# अध्याय 5 पुनर्प्राप्ति (पुनर्प्राप्ति)
 - [1. वेक्टर डेटाबेस खोज](#1. वेक्टर डेटाबेस खोज)
     - [1.1 समानता खोज](#1.1-समानता खोज)
     - [1.2 विविधता को संबोधित करना: अधिकतम सीमांत प्रासंगिकता (एमएमआर)] (#1.2 - विविधता को संबोधित करना: अधिकतम सीमांत प्रासंगिकता (एमएमआर))
     - [1.3 समाधान विशिष्टता: मेटाडेटा का उपयोग करें] (#1.3-समाधान विशिष्टता: मेटाडेटा का उपयोग करें)
     - [1.4 संबोधित विशिष्टता: मेटाडेटा में स्व-क्वेरी पुनर्प्राप्तिकर्ताओं का उपयोग करना] (#1.4 - विशिष्टता को संबोधित करना: मेटाडेटा में स्वयं-क्वेरी पुनर्प्राप्तिकर्ताओं का उपयोग करना)
     - [1.5 अन्य युक्तियाँ: संपीड़न](#1.5-अन्य युक्तियाँ: संपीड़न)
 - [2. विभिन्न प्रौद्योगिकियों का संयोजन](#2. विभिन्न प्रौद्योगिकियों का संयोजन)
 - [3. अन्य प्रकार की खोजें](#3. अन्य प्रकार की खोजें)

खोज हमारी खोज संवर्धित पीढ़ी (आरएजी) प्रक्रिया के केंद्र में है।

आइए पिछले पाठ में संग्रहीत वेक्टर डेटाबेस (`वेक्टरडीबी`) प्राप्त करें।

## 1. वेक्टर डेटाबेस पुनर्प्राप्ति

वर्तमान फ़ोल्डर में `OPENAI_API_KEY = "sk-..."` सामग्री के साथ एक नई `.env` फ़ाइल बनाएं

इस अध्याय के लिए `लार्क` पैकेज की आवश्यकता है, इसे स्थापित करने के लिए निम्न कमांड चलाएँ

In [1]:
!pip install -Uq lark

In [3]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']


### 1.1 समानता खोज

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/cs229_lectures/'
persist_directory_chinese = 'docs/chroma/matplotlib/'

पिछले पाठ में सहेजे गए वेक्टर डेटाबेस (`वेक्टरडीबी`) को लोड करें

In [5]:
embedding = OpenAIEmbeddings()

In [6]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [7]:
print(vectordb._collection.count())

209


In [8]:
vectordb_chinese = Chroma(
    persist_directory=persist_directory_chinese,
    embedding_function=embedding
)

In [9]:
print(vectordb_chinese._collection.count())

27


आइए अब अधिकतम सीमांत सहसंबंध का उदाहरण देखें। इसलिए, हम नीचे दिए गए उदाहरण से मशरूम के बारे में जानकारी लोड करेंगे।

आइए अब इसे एमएमआर के साथ चलाएं। आइए k को 2 के बराबर पास करें। हम अभी भी दो दस्तावेज़ वापस करना चाहते हैं, लेकिन आइए फ़ेच k को 3 के बराबर सेट करें, जहाँ हम शुरू में सभी तीन दस्तावेज़ लाते हैं। अब हम देख सकते हैं कि हमारे द्वारा पुनर्प्राप्त दस्तावेज़ में ज़हरीली जानकारी वापस आ गई थी।

In [10]:
texts = [
"""अमानिटा फालोइड्स में एक बड़ा और प्रभावशाली एपिजियस (जमीन के ऊपर) फलने वाला शरीर (बेसिडियोकार्प) होता है।""",
"""बड़े फलने वाले शरीर वाला मशरूम अमानिटा फालोइड्स है। कुछ किस्में पूरी तरह से सफेद होती हैं।""",
"""ए. फालोइड्स, उर्फ ​​डेथ कैप, सभी ज्ञात मशरूमों में से सबसे जहरीला है।""",
]

In [11]:
texts_chinese = [
"""अमनिता फालोइड्स में जमीन के ऊपर एक बड़ा और आकर्षक (एपिजियस) फलने वाला शरीर (बेसिडियोकार्प) होता है""",
"""बड़े फलने वाले शरीर वाला मशरूम अमनिटा फालोइड्स है। कुछ किस्में पूरी तरह से सफेद होती हैं।""",
"""ए. फालोइड्स, जिसे डेथ कैप भी कहा जाता है, सभी ज्ञात मशरूमों में सबसे जहरीला है।""",
]

इस उदाहरण के लिए, हम एक छोटा डेटाबेस बनाएंगे जिसे हम एक उदाहरण के रूप में उपयोग कर सकते हैं।

In [12]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

100%|██████████| 1/1 [00:00<00:00,  1.28it/s]


In [13]:
smalldb_chinese = Chroma.from_texts(texts_chinese, embedding=embedding)

100%|██████████| 1/1 [00:00<00:00,  2.30it/s]


यहां वे प्रश्न हैं जो हम इस उदाहरण के लिए पूछते हैं

In [14]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [15]:
question_chinese = "告诉我关于具有大型子实体的全白色蘑菇的信息"

अब, हम केवल दो सबसे प्रासंगिक दस्तावेज़ वापस करने के लिए, k=2 सेट करके एक समानता खोज चला सकते हैं।

हम देख सकते हैं कि इसमें इस बात का जिक्र तक नहीं है कि यह जहरीला है.

In [16]:
smalldb.similarity_search(question, k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.', metadata={}),
 Document(page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).', metadata={})]

In [17]:
smalldb_chinese.similarity_search(question_chinese, k=2)

[Document(page_content='一种具有大型子实体的蘑菇是毒鹅膏菌（Amanita phalloides）。某些品种全白。', metadata={}),
 Document(page_content='毒鹅膏菌（Amanita phalloides）具有大型且引人注目的地上（epigeous）子实体（basidiocarp）', metadata={})]

अब, अधिकतम सीमांत प्रासंगिकता (MMR) चलाते हैं।

k=2 सेट करें क्योंकि हम अभी भी दो दस्तावेज़ वापस करना चाहते हैं। Fetch_k=3 सेट करें, Fetch_k वे सभी दस्तावेज़ हैं जिन्हें हमने शुरू में प्राप्त किया था (3)।

In [18]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.', metadata={}),
 Document(page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.', metadata={})]

In [19]:
smalldb_chinese.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(page_content='一种具有大型子实体的蘑菇是毒鹅膏菌（Amanita phalloides）。某些品种全白。', metadata={}),
 Document(page_content='A. phalloides，又名死亡帽，是已知所有蘑菇中最有毒的一种。', metadata={})]

अब हम देख सकते हैं कि हमारे द्वारा पुनर्प्राप्त दस्तावेज़ में ज़हरीली जानकारी वापस आ गई थी।

### 1.2 समाधान विविधता: अधिकतम सीमांत प्रासंगिकता (एमएमआर)

खोज परिणामों की विविधता को कैसे बढ़ाया जाए, यह प्रश्न उठाने के लिए हमने बस एक उदाहरण का उपयोग किया।
 
अधिकतम सीमांत प्रासंगिकता क्वेरी प्रासंगिकता और परिणाम विविधता के बीच दोनों दुनियाओं में सर्वश्रेष्ठ हासिल करने का प्रयास करती है।

आइए पिछले पाठ के एक उदाहरण पर वापस जाएं, जब हमने प्रश्न के अनुसार वेक्टर डेटाबेस पर समानता खोज की थी

हम पहले दो दस्तावेज़ों को देख सकते हैं और केवल पहले कुछ पात्रों को देख सकते हैं और देख सकते हैं कि वे समान हैं।

In [20]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

In [21]:
docs_ss[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [22]:
docs_ss[1].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [23]:
question_chinese = "Matplotlib是什么？"
docs_ss_chinese = vectordb_chinese.similarity_search(question_chinese,k=3)

In [24]:
docs_ss_chinese[0].page_content[:100]

'第⼀回：Matplotlib 初相识\n⼀、认识matplotlib\nMatplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种'

In [25]:
docs_ss_chinese[1].page_content[:100]

'第⼀回：Matplotlib 初相识\n⼀、认识matplotlib\nMatplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种'

नोट: `एमएमआर` का उपयोग करके परिणामों में अंतर।

In [26]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [27]:
docs_mmr_chinese = vectordb_chinese.max_marginal_relevance_search(question_chinese,k=3)

जब हम एमएमआर चलाने के बाद परिणाम प्राप्त करते हैं, तो हम देख सकते हैं कि पहला वाला पिछले वाले जैसा ही है क्योंकि वह सबसे समान है।

In [28]:
docs_mmr[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [29]:
docs_mmr_chinese[0].page_content[:100]

'第⼀回：Matplotlib 初相识\n⼀、认识matplotlib\nMatplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种'

लेकिन जब हम दूसरे पर पहुंचते हैं, तो हम देख सकते हैं कि यह अलग है।

इससे प्रतिक्रियाओं में कुछ विविधता प्राप्त हुई।

In [30]:
docs_mmr[1].page_content[:100]

'algorithm then? So what’s different? How come  I was making all that noise earlier about \nleast squa'

In [31]:
docs_mmr_chinese[1].page_content[:100]

'By Datawhale 数据可视化开源⼩组\n© Copyright © Copyright 2021.y轴分为左右两个，因此 tick1 对应左侧的轴； tick2 对应右侧的轴。\nx轴分为上下两个'

### 1.3 विशिष्टताओं को संबोधित करना: मेटाडेटा का उपयोग करना

पिछले पाठ में, हमने एक प्रश्न दिखाया था जिसमें दस्तावेज़ में एक व्याख्यान के बारे में प्रश्न पूछा गया था, लेकिन प्राप्त परिणामों में अन्य व्याख्यानों के परिणाम भी शामिल थे।

इस समस्या को हल करने के लिए, कई वेक्टर डेटाबेस `मेटाडेटा` पर संचालन का समर्थन करते हैं।

`मेटाडेटा` प्रत्येक एम्बेडेड खंड के लिए संदर्भ प्रदान करता है।

In [32]:
question = "what did they say about regression in the third lecture?"

In [33]:
question_chinese = "他们在第二讲中对Figure说了些什么？"  

अब, हम इस समस्या को मैन्युअल रूप से हल करते हैं, हम एक मेटाडेटा फ़िल्टर `फ़िल्टर` निर्दिष्ट करेंगे

In [34]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"docs/cs229_lectures/MachineLearning-Lecture03.pdf"}
)

In [35]:
docs_chinese = vectordb_chinese.similarity_search(
    question_chinese,
    k=3,
    filter={"source":"docs/matplotlib/第二回：艺术画笔见乾坤.pdf"}
)

इसके बाद, हम देख सकते हैं कि परिणाम संबंधित अध्यायों से आते हैं

In [36]:
for d in docs:
    print(d.metadata)

{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 0}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 14}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 4}


In [35]:
for d in docs_chinese:
    print(d.metadata)
    

{'source': 'docs/matplotlib/第二回：艺术画笔见乾坤.pdf', 'page': 9}
{'source': 'docs/matplotlib/第二回：艺术画笔见乾坤.pdf', 'page': 10}
{'source': 'docs/matplotlib/第二回：艺术画笔见乾坤.pdf', 'page': 0}


बेशक, हम इस समस्या को हर बार मैन्युअल रूप से हल नहीं कर सकते, यह पर्याप्त स्मार्ट नहीं होगा।

अगला भाग दिखाएगा कि एलएलएम के माध्यम से इस समस्या को कैसे हल किया जाए

### 1.4 विशिष्टताओं को संबोधित करना: मेटाडेटा में स्व-क्वेरी रिट्रीवर्स का उपयोग करना

हमारे सामने एक दिलचस्प चुनौती है: हम अक्सर क्वेरी से ही मेटाडेटा का अनुमान लगाना चाहते हैं।

इस समस्या को हल करने के लिए, हम SelfQueryRetriever का उपयोग कर सकते हैं, जो निकालने के लिए LLM का उपयोग करता है:
 
1. क्वेरी (`क्वेरी`) स्ट्रिंग का उपयोग वेक्टर खोज के लिए किया जाता है, यानी: प्रश्न
2. मेटाडेटा फ़िल्टर को एक साथ पारित किया जाना है

अधिकांश वेक्टर डेटाबेस मेटाडेटा फ़िल्टर का समर्थन करते हैं, इसलिए किसी नए डेटाबेस और इंडेक्स की आवश्यकता नहीं होती है।

In [38]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [39]:
llm = OpenAI(temperature=0)

`एट्रिब्यूटइन्फो` हमें मेटाडेटा में विभिन्न फ़ील्ड और उनकी संबंधित स्थिति निर्दिष्ट करने की अनुमति देता है।

मेटाडेटा में, हमारे पास केवल दो फ़ील्ड हैं, `स्रोत` और `पेज`।

हम प्रत्येक संपत्ति का नाम, विवरण और प्रकार विवरण भरेंगे।

यह जानकारी वास्तव में एलएलएम को दी जाएगी, इसलिए इसे यथासंभव विस्तार से वर्णित करने की आवश्यकता है।

In [40]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [41]:
metadata_field_info_chinese = [
    AttributeInfo(
        name="source",
        description="讲义来源于 `docs/matplotlib/第一回：Matplotlib初相识.pdf`, `docs/matplotlib/第二回：艺术画笔见乾坤.pdf`, or `docs/matplotlib/第三回：布局格式定方圆.pdf` 的其中之一",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="讲义的那一页",
        type="integer",
    ),
]

In [42]:
document_content_description = "Lecture notes"
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [43]:
document_content_description_chinese = "课堂讲义"
retriever_chinese = SelfQueryRetriever.from_llm(
    llm,
    vectordb_chinese,
    document_content_description_chinese,
    metadata_field_info_chinese,
    verbose=True
)

In [44]:
question = "what did they say about regression in the third lecture?"

In [45]:
question_chinese = "他们在第二讲中对Figure说了些什么？"  

जब आप पहली बार अगली पंक्ति निष्पादित करते हैं, तो आपको एक चेतावनी मिलेगी कि Prediction_and_parse को हटा दिया गया है। इसे सुरक्षित रूप से अनदेखा किया जा सकता है।

In [46]:
docs = retriever.get_relevant_documents(question)

/root/autodl-tmp/env/gpt/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='regression' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='source', value='docs/cs229_lectures/MachineLearning-Lecture03.pdf') limit=None


In [47]:
docs_chinese = retriever_chinese.get_relevant_documents(question_chinese)

query='Figure' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='source', value='docs/matplotlib/第二讲：艺术画解破.pdf') limit=None


In [48]:
for d in docs:
    print(d.metadata)

{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 14}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 0}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 10}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 10}


In [49]:
for d in docs_chinese:
    print(d.metadata)

### 1.5 अन्य युक्तियाँ: संपीड़न

पुनर्प्राप्त दस्तावेज़ों की गुणवत्ता में सुधार करने का दूसरा तरीका संपीड़न है।

किसी क्वेरी के लिए सबसे प्रासंगिक जानकारी बड़ी मात्रा में अप्रासंगिक पाठ वाले दस्तावेज़ों में छिपी हो सकती है।

आपके आवेदन के माध्यम से संपूर्ण दस्तावेज़ पास करने के परिणामस्वरूप एलएलएम कॉल अधिक महंगी हो सकती हैं और प्रतिक्रिया ख़राब हो सकती है।

प्रसंग संपीड़न इस समस्या को हल करने के लिए डिज़ाइन किया गया है।

In [50]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [51]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [52]:
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)  # 压缩器

In [53]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [54]:
compression_retriever_chinese = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb_chinese.as_retriever()
)

In [55]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

/root/autodl-tmp/env/gpt/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
----------------------------------------------------------------------------------------------------
Document 2:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
----------------------------------------------------------------------------------------------------
Document 3:

"And the student said, "Oh, it was the MATLAB." So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, and we'll actually have a short MATLAB tutorial in one o

In [56]:
question_chinese = "Matplotlib是什么？"
compressed_docs_chinese = compression_retriever_chinese.get_relevant_documents(question_chinese)
pretty_print_docs(compressed_docs_chinese)

Document 1:

Matplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种静态，动态，交互式的图表。
----------------------------------------------------------------------------------------------------
Document 2:

Matplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种静态，动态，交互式的图表。


अब जब हम प्रश्न पूछते हैं, तो परिणामी दस्तावेज़ को देखें

हम दो चीजें देख सकते हैं.

1. ये सामान्य दस्तावेज़ों की तुलना में बहुत छोटे होते हैं
2. अभी भी कुछ डुप्लिकेट हैं, ऐसा इसलिए है क्योंकि हुड के तहत हम सिमेंटिक सर्च एल्गोरिदम का उपयोग कर रहे हैं।

यह वह समस्या है जिसे हमने पहले इस पाठ्यक्रम में एमएमआर का उपयोग करके हल किया था।

यह इस बात का एक बेहतरीन उदाहरण है कि आप सर्वोत्तम संभव परिणाम प्राप्त करने के लिए विभिन्न तकनीकों को कैसे जोड़ सकते हैं।

## 2. विभिन्न प्रौद्योगिकियों का संयोजन

ऐसा करने के लिए, वेक्टर डेटाबेस से रिट्रीवर बनाते समय हम खोज प्रकार को एमएमआर पर सेट कर सकते हैं।

फिर हम प्रक्रिया को फिर से चला सकते हैं और देख सकते हैं कि हमें एक फ़िल्टर किया गया परिणाम सेट लौटाया गया है जिसमें कोई डुप्लिकेट जानकारी नहीं है।

In [57]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [58]:
compression_retriever_chinese = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb_chinese.as_retriever(search_type = "mmr")
)

In [59]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

"MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data. And it's sort of an extremely easy to learn tool to use for implementing a lot of learning algorithms."
----------------------------------------------------------------------------------------------------
Document 2:

"And the student said, "Oh, it was the MATLAB." So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, and we'll actually have a short MATLAB tutorial in one of the discussion sections for those of you that don't know it."


In [60]:
question_chinese = "Matplotlib是什么？"
compressed_docs_chinese = compression_retriever_chinese.get_relevant_documents(question_chinese)
pretty_print_docs(compressed_docs_chinese)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Request failed due to server shutdown {
  "error": {
    "message": "Request failed due to server shutdown",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'Request failed due to server shutdown', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Sun, 16 Jul 2023 05:28:06 GMT', 'Content-Type': 'application/json', 'Content-Length': '141', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'openai-model': 'text-davinci-003', 'openai-organization': 'user-xnghkpntwvm31crtmex7n2j0', 'openai-processing-ms': '1159', 'openai-version': '2020-10-01', 'strict-transport-security': 'max-age=15724800; includeSubDomains', 'x-ratelimit-limit-requests': '3000', 'x-ratelimit-limit-tokens': '250000', 'x-ratelimit-remaining-requests': '2999', 'x-ratelimit-remaining-tokens': '249744', 'x-ratelimit-reset-reques

Document 1:

Matplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种静态，动态，交互式的图表。


## 3. अन्य प्रकार की पुनर्प्राप्ति

यह ध्यान देने योग्य है कि दस्तावेजों को पुनः प्राप्त करने के लिए vetordb एकमात्र उपकरण नहीं है।

`लैंगचेन` रिट्रीवर एब्स्ट्रैक्शन में दस्तावेज़ पुनर्प्राप्त करने के अन्य तरीके शामिल हैं, जैसे `टीएफ-आईडीएफ` या `एसवीएम`।

In [61]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [62]:
#पीडीएफ लोड करें
loader = PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text = [p.page_content for p in pages]
joined_page_text = " ".join(all_page_text)

# पाठ को विभाजित करें
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)


In [77]:
#पीडीएफ लोड करें
loader_chinese = PyPDFLoader("docs/matplotlib/第一回：Matplotlib初相识.pdf")
pages_chinese = loader_chinese.load()
all_page_text_chinese = [p.page_content for p in pages_chinese]
joined_page_text_chinese = " ".join(all_page_text_chinese)

# पाठ को विभाजित करें
text_splitter_chinese = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits_chinese = text_splitter_chinese.split_text(joined_page_text_chinese)

In [64]:
# पुनः प्राप्त करना
svm_retriever = SVMRetriever.from_texts(splits, embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [66]:
question = "What are major topics for this class?"  # 这门课的主要主题是什么？
docs_svm = svm_retriever.get_relevant_documents(question)
docs_svm[0]

Document(page_content="let me just check what questions you have righ t now. So if there are no questions, I'll just \nclose with two reminders, which are after class today or as you start to talk with other \npeople in this class, I just encourage you again to start to form project partners, to try to \nfind project partners to do your project with. And also, this is a good time to start forming \nstudy groups, so either talk to your friends  or post in the newsgroup, but we just \nencourage you to try to star t to do both of those today, okay? Form study groups, and try \nto find two other project partners.  \nSo thank you. I'm looking forward to teaching this class, and I'll see you in a couple of \ndays.   [End of Audio]  \nDuration: 69 minutes", metadata={})

In [67]:
question = "what did they say about matlab?"  # 他们关于Matlab说了些什么？
docs_tfidf = tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]

Document(page_content="Saxena and Min Sun here did, wh ich is given an image like this, right? This is actually a \npicture taken of the Stanford campus. You can apply that sort of cl ustering algorithm and \ngroup the picture into regions. Let me actually blow that up so that you can see it more \nclearly. Okay. So in the middle, you see the lines sort of groupi ng the image together, \ngrouping the image into [inaudible] regions.  \nAnd what Ashutosh and Min did was they then  applied the learning algorithm to say can \nwe take this clustering and us e it to build a 3D model of the world? And so using the \nclustering, they then had a lear ning algorithm try to learn what the 3D structure of the \nworld looks like so that they could come up with a 3D model that you can sort of fly \nthrough, okay? Although many people used to th ink it's not possible to take a single \nimage and build a 3D model, but using a lear ning algorithm and that sort of clustering \nalgorithm is the first ste

In [78]:
# चीनी संस्करण
svm_retriever_chinese = SVMRetriever.from_texts(splits_chinese, embedding)
tfidf_retriever_chinese = TFIDFRetriever.from_texts(splits_chinese)

In [79]:
question_chinese = "这门课的主要主题是什么？" 
docs_svm_chinese = svm_retriever_chinese.get_relevant_documents(question_chinese)
docs_svm_chinese[0]

Document(page_content='fig, ax = plt.subplots()  \n# step4 绘制图像，  这⼀模块的扩展参考第⼆章进⼀步学习\nax.plot(x, y, label=\'linear\')  \n# step5 添加标签，⽂字和图例，这⼀模块的扩展参考第四章进⼀步学习\nax.set_xlabel(\'x label\') \nax.set_ylabel(\'y label\') \nax.set_title("Simple Plot")  \nax.legend() ;\n思考题\n请思考两种绘图模式的优缺点和各⾃适合的使⽤场景\n在第五节绘图模板中我们是以 OO 模式作为例⼦展示的，请思考并写⼀个 pyplot 绘图模式的简单模板', metadata={})

In [80]:
question_chinese = "Matplotlib是什么？"
docs_tfidf_chinese = tfidf_retriever_chinese.get_relevant_documents(question_chinese)
docs_tfidf_chinese[0]

Document(page_content='fig, ax = plt.subplots()  \n# step4 绘制图像，  这⼀模块的扩展参考第⼆章进⼀步学习\nax.plot(x, y, label=\'linear\')  \n# step5 添加标签，⽂字和图例，这⼀模块的扩展参考第四章进⼀步学习\nax.set_xlabel(\'x label\') \nax.set_ylabel(\'y label\') \nax.set_title("Simple Plot")  \nax.legend() ;\n思考题\n请思考两种绘图模式的优缺点和各⾃适合的使⽤场景\n在第五节绘图模板中我们是以 OO 模式作为例⼦展示的，请思考并写⼀个 pyplot 绘图模式的简单模板', metadata={})